Есть датасет:
- Из логов были взяты пары ([запрос] - [объект на который кликнул пользователь]). Это положительные примеры (метка 1 в датасете)
- Для каждого запроса был подобран в пару негативный объект (метка 0) следующим образом: определяем к какой рубрике относится положительный пример; выбираем случайный объект из другой рубрики. Идея в том, что этот пример маловероятно будет релевантным.

На этих данных, используя кросс-валидацию, обучались различные модели. Метрики качества были хорошими.
При попытки тестирования на реальных данных, качество моделей сильно уступало тестовым метрикам. 


Задача: выявить особенности датасета, которые приводили к данным результатам и объяснить почему так происходило.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('clicks_dataset_msk_20230101_20230725_spec.csv', names = ['Запрос', 'Объект','Метка'])
df

,Запрос,Объект,Метка
0,1590973,168299,1
1,1590973,718560,0
2,1234953,325828,1
3,1234953,135968,0
4,3326557,334526,1
...,...,...,...
12979877,493850,545216,0
12979878,3008578,1128285,1
12979879,3008578,738151,0
12979880,4481384,413671,1


In [3]:
df.isna().sum()

Запрос    0
Объект    0
Метка     0
dtype: int64

Пропусков нет.

In [4]:
df.drop('Запрос', axis= 1 , inplace= True )
df

,Объект,Метка
0,168299,1
1,718560,0
2,325828,1
3,135968,0
4,334526,1
...,...,...
12979877,545216,0
12979878,1128285,1
12979879,738151,0
12979880,413671,1


Убрали столбец "Запрос", так как он не несет смысловой нагрузки в обучении модели.

In [5]:
# Посмотрим дубли
df.duplicated().sum()

11514553

In [6]:
# Удалим дубли
df.drop_duplicates(inplace=True)

df.shape

(1465329, 2)

In [7]:
df['Метка'].value_counts()

1    1005313
0     460016
Name: Метка, dtype: int64

Классы не сбалансированы, поэтому качество моделей сильно уступало тестовым метрикам.

Есть три метода борьбы с дисбалансом:

- взвешивание классов;
- уменьшение выборки;
- увеличение выборки.

### Взвешивание классов

Необходимо придать объектам редкого класса больший вес. В алгоритмах логистической регрессии, решающего дерева и случайного леса в библиотеке sklearn есть аргумент class_weight. Если указать class_weight='balanced', алгоритм посчитает, во сколько раз класс 0 встречается чаще класса 1.

### Уменьшение выборки

Преобразование проходит в несколько этапов:
 - разделить обучающую выборку на отрицательные и положительные объекты;
 - случайным образом отбросить часть из отрицательных объектов;
 - с учётом полученных данных создать новую обучающую выборку;
 - перемешать данные. Положительные не должны идти следом за отрицательными: алгоритмам будет сложнее обучаться.

### Увеличение выборки

Преобразование проходит в несколько этапов:
 - разделить обучающую выборку на отрицательные и положительные объекты;
 - скопировать несколько раз положительные объекты;
 - с учётом полученных данных создать новую обучающую выборку;
 - перемешать данные: идущие друг за другом одинаковые вопросы не помогут обучению.

### Вывод: 

Если модель обучается на не сбалансированных данных,то она склонна к смещению в сторону более представленного класса, и это привело к плохим результатам на реальных данных.